In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import DataFrameMapper, gen_features, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from scipy import stats
import xgboost as xgb

In [6]:
df = pd.read_csv('data/train_cols.csv')

In [8]:
df['hour_of_day'] = df['hour'].map(lambda x: int(str(x)[-2:]))
df['day'] = df['hour'].map(lambda x: int(str(x)[4:6]))
df['month'] = df['hour'].map(lambda x: int(str(x)[2:4]))


In [9]:
df.columns

Index(['click', 'hour', 'C1', 'banner_pos', 'device_type', 'device_conn_type',
       'C15', 'C16', 'C18', 'hour_of_day', 'day', 'month'],
      dtype='object')

In [10]:
df.head()

,click,hour,C1,banner_pos,device_type,device_conn_type,C15,C16,C18,hour_of_day,day,month
0,0,14102100,1005,0,1,2,320,50,0,0,21,10
1,0,14102100,1005,0,1,0,320,50,0,0,21,10
2,0,14102100,1005,0,1,0,320,50,0,0,21,10
3,0,14102100,1005,0,1,0,320,50,0,0,21,10
4,0,14102100,1005,1,1,0,320,50,0,0,21,10


In [36]:
# mapper2 = DataFrameMapper([
#     (['C1'], LabelBinarizer()),
#     (['banner_pos'], LabelBinarizer()),
#     (['device_type'], LabelBinarizer()),
#     (['device_conn_type'], LabelBinarizer()),
#     (['C15'], LabelBinarizer()),
#     (['C16'], LabelBinarizer()),
#     (['C18'], LabelBinarizer()),
#     (['hour_of_day'], LabelBinarizer()),
#     (['day'], LabelBinarizer())
# ], input_df=True)

feature_def = gen_features(
    columns=['C1', 'banner_pos', 'device_type', 'device_conn_type',
       'C15', 'C16', 'C18', 'hour_of_day'],
    classes=[LabelBinarizer])
    
mapper = DataFrameMapper(feature_def, input_df=True)

In [39]:
test = df.iloc[0:15000]

In [40]:
mapper.fit_transform(test).shape

(15000, 33)

In [48]:
pipe = Pipeline([
    ('featurize', mapper),
    ('NB', MultinomialNB())])

In [ ]:
np.round(cross_val_score(pipe, X=df, y=df.click, scoring='f1'), 2)

In [ ]:
pipeline = Pipeline([('featurize', featurize(features)), ('forest', RandomForestClassifier(n_estimators = 10))])

pipeline.fit(X, y).predict(df_train)

param_grid = dict(forest__n_estimators = [2, 16, 32], forest__criterion = ['gini', 'entropy'])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, scoring='accuracy')

best_pipeline = grid_search.fit(X, y).best_estimator_
best_pipeline.get_params()['forest']
